In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 

import torch

import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision.transforms as transforms
from torch.utils.data import  Dataset, TensorDataset, DataLoader


import lib.pytorch_trainer as ptt

from src.imgnet_utils import denormalize

from src.data_loader import _create_dataLoader

from src.Dataset import KaggleSafeDriverDataset

from src.plot_utils import (plot_classes, plot_distribution,
                            statistical_analysis_image, classDistribution,
                            plot_metrics,visualize_predictions,
                            plot_cm_train_valid,plot_layers_weight)
      
from src.convnet_models import (MyResNet, MyInception, MyDenseNet)
from src.extractor_utils import (predict, getPrediction,save_prediction,
                                 load_prediction, torch_summarize,RandomSearch)

from utils.utils import (create_submission ,metrics2csv)

# Checking for resources avaliables

In [2]:
!free -h
!nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:           220G         20G        138G        1.3G         61G        197G
Swap:            0B          0B          0B
Sat Dec  2 23:07:51 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.90                 Driver Version: 384.90                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00003609:00:00.0 Off |                    0 |
| N/A   47C    P0   144W / 149W |   1599MiB / 11439MiB |     26%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00004BDC:0

In [3]:
print("{} GPU's available:".format(torch.cuda.device_count()) )
cpu_count = torch.multiprocessing.cpu_count()
print("\ncpu_count: {}".format(cpu_count))

4 GPU's available:

cpu_count: 24


In [4]:
use_gpu = True
use_DataParalel= True
use_CPU= False       

if use_gpu:
    if use_DataParalel: 
        print("Using DataParalel in all {} GPUS".format(torch.cuda.device_count()))
    else:
        print('Using only one GPU') 
if use_CPU:         # http://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html 
    print("Using {} CPU's".format(cpu_count))

Using DataParalel in all 4 GPUS



# Path to features

In [5]:
path2features= "./features/" 

# RandomSearch on hyperparameters

In [6]:
# Flag to optimze hyperparameters
hiper_tunning = True

# Choosing Model

In [7]:
use_resnet = False
use_inception = False
use_denseNet = True

if use_resnet:
    print('Using ResNet model')
    model_name = "ResNet"
    model = MyResNet()
elif use_inception:
    print('Using Inception model')
    model_name = "Inception"
    model = MyInception()
elif use_denseNet:
    print('Using DenseNet model')
    model_name = "DenseNet"    
    model = MyDenseNet()  

Using DenseNet model


In [8]:
if use_gpu:
    if use_DataParalel:
        print("Using all GPU's ")
        model.mrnc = torch.nn.DataParallel(model.mrnc) #device_ids=[1,3]
        model.mrnc = model.mrnc.cuda()
        model.mrnd = torch.nn.DataParallel(model.mrnd) #device_ids=[1,3]
        model.mrnd = model.mrnd.cuda()
    else:
        print('Using GPU')# {}'.format(device_id))
        model.cuda()
else:
    print("Using CPU's")

Using all GPU's 


# Loading features

In [9]:
model_name

'DenseNet'

In [10]:
load_feat= load_prediction(path2features,model_name)

Loaded features with shapes: 


train:
pred torch.Size([17940, 2208]), true torch.Size([17940])

valid:
pred torch.Size([4484, 2208]), true torch.Size([4484])

test:
pred torch.Size([79726, 2208]), true torch.Size([79726])


In [11]:
load_feat['train'][0].shape,load_feat['train'][1].shape

(torch.Size([17940, 2208]), torch.Size([17940]))

In [12]:
conv_dset ={
'train': TensorDataset(load_feat['train'][0], load_feat['train'][1]),
'valid': TensorDataset(load_feat['valid'][0], load_feat['valid'][1]),
'test': TensorDataset(load_feat['test'][0], load_feat['test'][1])
}

# Creating Dataloaders

In [13]:
batch_size= 32

dset_loaders_convnet = _create_dataLoader(conv_dset, batch_size, 
                        pin_memory=False, use_shuffle= True)

In [14]:
dset_convnet_sizes = {x: len(dset_loaders_convnet[x]) for x in ['train','valid', 'test']} 
dset_convnet_sizes

{'test': 2492, 'train': 561, 'valid': 141}

# Trainning Model

In [15]:
if hiper_tunning: 
    path2saveModel = './models/'+model_name+'RandomSearch'+'DistractDriver' 
else:
    path2saveModel = './models/'+model_name+'DistractDriver' 

savebest = ptt.ModelCheckpoint(path2saveModel,reset=True, verbose=0)

In [16]:
loss_fn = nn.CrossEntropyLoss()
num_epochs = 2

optimizer =  optim.Adam(model.mrnd.parameters(), lr=1e-3,weight_decay=0)

params = {'model' : model.mrnd, 
    'criterion': loss_fn,  
    'optimizer': optimizer, 
    'callbacks': [savebest, ptt.AccuracyMetric()] #ptt.PlotCallback(),
}

In [17]:
# Search space for tunning hyperparameters 
args= {'lr':[1e-5,1e-2],
       'weight_decay':[1e-8,1e-3] }

In [18]:
if hiper_tunning: 
    
    
    (history,best_result,
     best_parameters,best_model) = RandomSearch(params,args,
                                                num_epochs=2,path2saveModel=path2saveModel,
                                                dset_loaders_convnet=dset_loaders_convnet,
                                                MAX_IT=2,verbose=1)
    
    
    print('\nBest parameters {}'.format(best_parameters))
    print('\nBest result {}'.format(best_result))
    
else:
    params['callbacks'].append(ptt.PrintCallback())
    scheduler = StepLR(optimizer, step_size=5, gamma=0.55)
    trainer = ptt.DeepNetTrainer(use_gpu=use_gpu,lr_scheduler=scheduler,**params)
    trainer.fit_loader(num_epochs, dset_loaders_convnet['train'], dset_loaders_convnet['valid'])

Iteration 0/2
Best model was saved at epoch 2 with loss 0.99647: ./models/DenseNetRandomSearchDistractDriver
train_loss: 0.9043433961793863, val_loss 0.9964718738908113
Iteration 1/2
Best model was saved at epoch 2 with loss 0.33047: ./models/DenseNetRandomSearchDistractDriver
train_loss: 0.25416610095413233, val_loss 0.33047048233110493
Best parameters {'lr': 0.00670084188558765, 'weight_decay': 0.00051358144486028298}

Best result [0.33047048233110493, 0.25416610095413233, <lib.pytorch_trainer.DeepNetTrainer object at 0x7fd92029c208>]


In [ ]:
#print(torch_summarize(model))

# Loading best model

In [ ]:
trainer.load_state(path2saveModel)
model.mrnd = trainer.model

In [ ]:
metrics2csv(trainer, model_name)

In [ ]:
train_eval = trainer.evaluate_loader(dset_loaders_convnet['train'])
valid_eval = trainer.evaluate_loader(dset_loaders_convnet['valid'])

In [ ]:
train_eval, valid_eval

# Dataset without shuffling 

#### Utilized to plot mismatch cases

In [ ]:
path2train = "/mnt/home/e209440/data/train"  

In [ ]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std  = np.array([0.229, 0.224, 0.225])

img_width = img_height=300 #to use InceptionV3 it must img_width and img_height be changed to 300

# Data augmentation and normalization for training 
data_transforms = {
    'valid': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
}   

In [ ]:
batch_size = 32
use_only = 1.0 # Use only is the percentage of the full dataset

In [ ]:
dsets = {
    'valid': KaggleSafeDriverDataset(path2train,
                                     transforms=data_transforms['valid'],
                                     use_only=use_only, is_val=True, val_size=0.2),
}

In [ ]:

dset_loaders_wshuffle = _create_dataLoader(dsets, batch_size,
                                           pin_memory=False, use_shuffle= False)

# Evaluating resultings 

In [ ]:
#we need to use all dloader, because this one has use_shuffle false
result_train = predict(dset_loaders_convnet['train'], model.mrnd, use_gpu=use_gpu)

result_valid = predict(dset_loaders_wshuffle['valid'], model, use_gpu=use_gpu)

result_test = predict(dset_loaders_convnet['test'], model.mrnd, use_gpu=use_gpu)

In [ ]:
predictions_out = {'train': (result_train['pred'], result_train['true'], model_name),
                   'valid': (result_valid['pred'], result_valid['true'], model_name),
                   'test': (result_test['pred'], result_test['true'], model_name)}

In [ ]:
path2results = './results/'

In [ ]:
save_prediction(path2results,predictions_out)

In [ ]:
result_train = getPrediction(result_train)
result_valid = getPrediction(result_valid)
# result_test['pred'] must be an array of probabilities to make the submission

In [ ]:
correct_train = (result_train['true'] == result_train['pred']).sum()
correct_valid = (result_valid['true'] == result_valid['pred']).sum()

In [ ]:
print('Train: ', correct_train, '/', '17940' )# len(dsets['train'])
print('Valid: ', correct_valid, '/', '4484' ) # len(dsets['valid'])

# Make submission of the Test set

In [ ]:
create_submission(result_test, model_name + '_distracted_driver')